# Natural Language Processing with Disaster Tweets

##  所需要的库

```python
import re
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

```

##  数据的预处理

> 1. 导入数据
>
>    ```python
>    train_df = pd.read_csv('../data/Natural Language Processing with Disaster Tweets/train.csv')
>    test_df = pd.read_csv('../data/Natural Language Processing with Disaster Tweets/test.csv')
>    ```

> 2. 数据清洗
>
>    ```python
>    def clean_text(text):
>        text= text.lower()
>        text= re.sub('[0-9]', '', text)
>        text= re.sub('#', '', text)  
>        text= re.sub('-', '', text)  
>        text= re.sub('()', '', text) 
>        text= re.sub('=>', '', text) 
>        text= re.sub('|', '', text) 
>        text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
>        return text
>    # 该数据还能进行更多的数据清洗，这边只是做了一个范例。
>    ```
>
>    ```python
>    # 将数据清洗的操作应用到数据中，并将数据进行保存。
>    train_df["text"] = train_df["text"].apply(clean_text)
>    test_df["text"] = test_df["text"].apply(clean_text)
>    train_df.to_csv('../data/Natural Language Processing with Disaster Tweets/train_clean.csv', index=None)
>    test_df.to_csv('../data/Natural Language Processing with Disaster Tweets/test_clean.csv', index=None)
>    ```

## 用BERT进行fine-tuning

> 1. 模型选择
>
>    ```python
>    # 本次用蒸馏过的bert模型来进行微调，相较于原本的bert，体型更小并且精度基本没有损失。
>    # 其中uncased表示按照小写字母来进行处理。
>    tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
>    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
>    ```

> 2. 数据集的加载
>
>    ```python
>    # 这里用huggingface的dataset包来进行数据读取。
>    train_datasets = load_dataset('csv', data_files={'train': '../data/Natural Language Processing with Disaster Tweets/train_clean.csv'})
>    test_datasets = load_dataset('csv', data_files={'test': '../data/Natural Language Processing with Disaster Tweets/test_clean.csv'})
>    
>    # 删除了一些无关的列属性，也可以将这些列属性加入文本来进行训练
>    train_datasets = train_datasets.remove_columns(['id', 'keyword', 'location'])
>    test_id = list(test_datasets['test']['id']) # 将test_id存起来，方便后面生成提交的文件
>    test_datasets = test_datasets.remove_columns(['id', 'keyword', 'location'])
>    
>    # 这个操作可以将训练集分出一部分当作验证集，来进行超参数选择，本次不进行这个，直接进行训练和预测
>    # train_datasets = train_datasets['train'].train_test_split(test_size=0.1)
>    ```

> 3. tokenizer进行bert输入层操作
>
>    ```python
>    # 选取最长长度为64，不足的补0，超过的进行截断
>    def tokenize_function(examples):
>        return tokenizer(examples["text"], padding='max_length', max_length=64, truncation=True)
>    ```
>
>    ```python
>    # 将训练集和测试集进行token操作，返回的结果会多出attention_mask、input_ids和已经进行编码的text，可自行打印
>    train_datasets = train_datasets.map(tokenize_function, batched=True)
>    test_datasets = test_datasets.map(tokenize_function, batched=True)
>    ```

> 4. 准确率计算函数
>
>    ```python
>    # 该准确率计算函数要使用可能需要有验证集
>    metric =load_metric("../metrics/accuracy/accuracy.py")# 该函数可能需要下载到本地才能进行使用。
>    def compute_metrics(eval_pred):
>        logits, labels = eval_pred
>        predictions = np.argmax(logits, axis=-1)
>        return metric.compute(predictions=predictions, references=labels)
>    ```

> 5. 模型定义和超参数选择
>
>    ```python
>    # TrainingArguments还有很多其他的属性，建议可以去看官方的文档
>    # 由于是直接进行训练，没有用验证集，故将一些属性注释掉
>    training_args = TrainingArguments(
>        output_dir='./results',         	# 模型结果的保存路径       
>        num_train_epochs=3,             	# 模型训练次数
>        per_device_train_batch_size=64, 	# 每个显卡上训练的batch大小
>        #per_device_eval_batch_size = 16,   # 每个显卡上验证的batch大小
>        warmup_steps=500,                   # 热学习率
>        weight_decay=0.01,             		# 权重衰退
>        #evaluation_strategy = "epoch",     # 评价模型方式
>        save_strategy = "epoch",            # 保存模型方式
>        learning_rate=2e-5,					# 学习率
>        #load_best_model_at_end=True,		# 是否在学习结束后加载最好模型
>        #metric_for_best_model='accuracy'   # 评价最好模型标准
>        logging_steps = 100					# 打印log的操作步数
>    )
>    
>    trainer = Trainer(
>        model=model,                        
>        args=training_args,             
>        train_dataset=train_datasets['train'], 
>        #eval_dataset=train_datasets['test'],  # 这是之前分的验证集
>        #compute_metrics=compute_metrics       # 这是之前定义的计算准确率的函数
>    )
>    
>    # 将这些超参数进行定义后，就可以直接开始训练
>    # pytorch可直接调用api，但是其他的需要参考文档
>    trainer.train()
>    ```

> 6. 预测
>
>    ```python
>    output = trainer.predict(test_dataset=test_datasets['test']) # 进行预测
>    predictions = torch.argmax(torch.from_numpy(output[0]), dim=-1) # 对预测结果取概率最大的标签，可以打印进行查看
>    
>    # 这是生成kaggle提交的结果
>    submission = pd.DataFrame({'id':test_id, 'target':list(predictions.numpy())})
>    submission.to_csv('submission.csv', index=False)
>    ```

## 结果

本次kaggle的score为0.81520，但是受限于设备和其他原因，应该还是可以从以下进行改进

> 1. 数据清洗这块，只是粗看了一些较多的噪声，可以进行进一步处理，增加数据的质量。
> 2. 本次只使用了text这个属性，keyword和location并没有使用，可以进行拼接或者其他操作来提高准确率。
> 3. 由于训练是租用云gpu，所以很多参数并没有好好调整，包括预训练模型的选择也只是选择了一个很常用的模型，有条件的可以选择一个更大的模型并且参数可以进行更进一步调整。
> 4. 本代码可能不一定能直接跑起来，主要注意各种文件的路径，如何预训练模型下不下来，可以去https://huggingface.co/models 进行下载，主要下载config、pytorch_model、vocab这三个文件，放在一个文件夹即可。

